In [223]:
import os
import face_recognition
import numpy as np
import pandas as pd
from skimage import io
from helper.face_detection import detect_faces
from PIL import Image
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

from helper.image_clustering import image_clustering
%matplotlib inline

# Prepare Image Used

In [230]:
import pathlib
source_file = pd.read_csv("../data/train.csv")
training_path = "../data/raw/Training/{}"
data_observation_id = 519
observation_folder_path = training_path.format(source_file.loc[data_observation_id]["nomor"])
filenames = []
for file in os.listdir(observation_folder_path):
    if pathlib.Path(file).suffix in ['.jpg', 'jpeg', '.png']:
        filenames.append(file)

In [225]:
images_path_inside_folder = ["../data/raw/Training/{}/{}".format(source_file.loc[data_observation_id]["nomor"], filename)  for filename in filenames]

In [226]:
images = [io.imread(image_path) for image_path in images_path_inside_folder ]
len(images)

ValueError: Could not find a format to read the specified file in single-image mode

In [ ]:
faces_detected = [(n, detect_faces(img)) for n, img in enumerate(images)]
faces_detected

In [ ]:
extracted_face_detected = []

for item in faces_detected:
    (img_index, face_rect) = item
    if len(face_rect) > 1:
        for face in face_rect:
             extracted_face_detected.append((img_index, face))
    else:
        extracted_face_detected.append((img_index, face_rect[0]))

extracted_face_detected[0]

In [ ]:
list_face_image = []
for n, (img_index, face_rect) in enumerate(extracted_face_detected):
    face = Image.fromarray(images[img_index]).crop(face_rect)
    list_face_image.append(face)
    plt.subplot(1, len(extracted_face_detected), n+1)
    plt.axis('off')
    plt.imshow(face)

In [ ]:
len(list_face_image)

# Clustering Image

In [ ]:
# encode image to array
list_encoded_face = [face_recognition.face_encodings(np.asarray(img)) for img in list_face_image]
# list_encoded_face_filtered = [np.array(filtered[0]) for filtered in list(filter(lambda encode: len(encode) > 0, list_encoded_face))]
list_encoded_face_filtered = [filtered[0] if len(filtered) else np.random.rand(128) for filtered in list_encoded_face]
# np.array(list_encoded_face_filtered[0]).shape

In [ ]:
k_mean_cluster = KMeans(n_clusters=len(list_encoded_face_filtered)//2)
k_mean_cluster.fit(list_encoded_face_filtered)
k_mean_cluster.labels_

In [ ]:
dbscan_cluster = DBSCAN(min_samples = 2)
dbscan_cluster.fit(list_encoded_face_filtered)
dbscan_cluster.labels_

In [ ]:
agglomerative_cluster = AgglomerativeClustering()
agglomerative_cluster.fit(list_encoded_face_filtered)
agglomerative_cluster.labels_


In [ ]:
# mode_label = stats.mode( np.array(dbscan_cluster.labels_)[dbscan_cluster.labels_ != -1])[0][0]
np.count_nonzero(dbscan_cluster.labels_ != -1) > 0

In [ ]:
filtered_label = [label == mode_label and mode_label != -1 for label in dbscan_cluster.labels_]
filtered_label

In [ ]:
main_person_face = np.array(list_face_image)[filtered_label]
main_person_face

In [ ]:
for n, img_face in enumerate(main_person_face):
    plt.subplot(1, len(main_person_face), n+1)
    plt.axis('off')
    plt.imshow(img_face)

In [ ]:
from helper.folder_image_loader import load_image_folder


list_face_image_from_folder = load_image_folder(520)
main_face = image_clustering(list_face_image_from_folder)
for n, img_face in enumerate(main_face):
    plt.subplot(1, len(main_face), n+1)
    plt.axis('off')
    plt.imshow(img_face)